In [ ]:
# HOUSE RULES
SPLITTING_SAME_VALUE = True # if you can split same-valued but different-ranked cards


In [ ]:
class Game:
    def __init__(self, d, number_decks=6):
        self.deck = Deck(number_decks)
        self.deck.shuffle()
        self.d = d
        self.d.hit_button.on_click(self.hit)
        self.d.stand_button.on_click(self.stand)
        self.d.double_button.on_click(self.double_down)
        self.d.split_button.on_click(self.split)

        print(f'New game with {self.deck}')
        self.d.set_blackjack_chance(self.deck.chance_of_blackjack())
        self.d.update_deck_info(self.deck)

        self.payouts = [0]

    def dealer_draw_card(self):
        card = self.deck.deal()
        print(f"Dealer draws {card}")
        
        self.dealer.add_card(card)
        self.d.update_deck_info(self.deck)
        self.d.dealer_cards_label.value = str([str(card) for card in self.dealer.cards])
        self.d.dealer_value_label.value = str(self.dealer.value)
        self.d.set_blackjack_chance(self.deck.chance_of_blackjack())

    
    def player_draw_card(self):
        card = self.deck.deal()
        print(f"Player draws {card}")
        self.player.add_card(card)
        self.d.update_deck_info(self.deck)
        self.d.player_cards_label.value = str([str(card) for card in self.player.cards])
        self.d.player_value_label.value = str(self.player.value)
        if len(self.player.cards) >= 2:
            self.d.set_hit_bust_chance(self.chance_to_bust(self.player))
            self.d.set_blackjack_chance(self.deck.chance_of_blackjack())
            self.d.set_stand_win_chance(get_stand_chances(self.player, self.dealer, self.deck)[0])
        

    def player_turn(self, already_has_one_card=False):
        self.d.activation(0,1)
        if not already_has_one_card:
            self.player_draw_card()
        self.player_draw_card()

        if self.player.is_blackjack():
            self.d.player_cards_label.style.text_color = 'gold'
            self.d.player_cards_label.style.font_weight = 'bolder'
            print("Player has a Blackjack!")
            self.finish_round()

        if (self.player.cards[0].rank == self.player.cards[1].rank) or (SPLITTING_SAME_VALUE and self.player.cards[0].value == self.player.cards[1].value) and self.player_split_hand is None:
            print("Player can split!")
            self.d.split_button.disabled = False
        
        if self.player.value < 21:
            if self.player_split_hand is None:
                self.d.double_button.disabled = False
            self.d.hit_button.disabled = False
            self.d.stand_button.disabled = False

    def hit(self, b):
        self.d.disable_action_buttons()
        with self.d.output:
            self.player_draw_card()
            if self.player.value > 21:
                print("Player busts!")
                self.d.player_value_label.style.text_color = 'red'
                self.d.player_value_label.style.font_weight = 'bolder'
                self.finish_round()

            elif self.player.value == 21:
                print("Player stands at 21!")
                self.d.player_value_label.style.font_weight = 'bolder'
                self.finish_round()
        
            else:
                self.d.hit_button.disabled = False
                self.d.stand_button.disabled = False
            

    def stand(self, b):
        self.d.disable_action_buttons()
        with self.d.output:
            self.d.player_value_label.style.font_weight = 'bolder'
            print("Player stands at", self.player.value)
            self.finish_round()

    def double_down(self, b):
        self.d.disable_action_buttons()
        with self.d.output:
            print("Player doubles down!")
            self.player_doubled_down = True
            self.player_draw_card()
            if self.player.value > 21:
                print("Player busts!")
                self.d.player_value_label.style.text_color = 'red'
                self.d.player_value_label.style.font_weight = 'bolder'
            else:
                print(f"Player stands at {self.player.value}")
                self.d.player_value_label.style.font_weight = 'bolder'
            self.finish_round()
            
    
    def split(self, b):
        self.d.disable_action_buttons()
        with self.d.output:
            print("Player splits!")
            self.player_split_hand = Hand([self.player.cards[0]]) # create split hand to be played later
            self.player = Hand([self.player.cards[1]])
            print("Playing first split hand", self.player)
            self.player_draw_card()
            if self.player.value > 21:
                print("Player busts!")
                self.d.player_value_label.style.text_color = 'red'
                self.d.player_value_label.style.font_weight = 'bolder'
                self.finish_round()

            elif self.player.value == 21:
                print("Player stands at 21!")
                self.d.player_value_label.style.font_weight = 'bolder'
                self.finish_round()
        
            else:
                self.d.hit_button.disabled = False
                self.d.stand_button.disabled = False
           
    def finish_round(self):
        self.d.disable_action_buttons()

        if self.player_split_hand != None:
            self.other_hand = copy(self.player)
            self.player = self.player_split_hand
            self.player_split_hand = None
            print("Playing second split hand!", self.player)
            self.d.reset_colors()
            self.player_turn(already_has_one_card=True)
            return

        self.d.activation(1,0)
        self.dealer_turn()

        self.d.activation(0,0)

        
        if self.other_hand == None:
            payout = self.calculate_payout(self.player)
            if self.player_doubled_down:
                payout *= 2
            print(f'Payout: {payout}')
            self.payouts.append(payout)
        else:
            # split hands
            first_payout = self.calculate_payout(self.other_hand)
            second_payout = self.calculate_payout(self.player)
            print("Payout for first split hand", first_payout)
            print("Payout for the second split hand", second_payout)
            self.payouts.append(first_payout)
            self.payouts.append(second_payout)
        

        self.d.update_payout_graph(self.payouts)

        self.d.deal_button.disabled = False

        
    def dealer_first_card(self):
        self.d.activation(1,0)
        self.dealer_draw_card()
        #TODO: in non-european games, the dealer checks for blackjack here and the round stops if no other players have a blackjack.
        # however, this does noch change the odds of the game, only the number of cards dealt (if the dealer has a blackjack, it will beat all non-blackjack players anyway)


    def dealer_turn(self):
        self.d.activation(1,0)
        while self.dealer.value < 17:
            self.dealer_draw_card()
            if self.dealer.value > 21:
                self.d.dealer_value_label.style.text_color = 'red'
                self.d.dealer_value_label.style.font_weight = 'bolder'
                break
            elif self.dealer.value >= 17:
                if self.dealer.is_blackjack():
                    self.d.dealer_cards_label.style.text_color = 'gold'
                    self.d.dealer_cards_label.style.font_weight = 'bolder'
                    print("Dealer has a Blackjack!")
                else:
                    self.d.dealer_value_label.style.font_weight = 'bolder'
                    print("Dealer stands at", self.dealer.value)
                break

    def calculate_payout(self, player_hand):
        if player_hand.is_blackjack():
            if self.dealer.is_blackjack():
                print("Player and Dealer have a Blackjack!")
                print("Push!")
                return 0
            else: 
                self.d.dealer_value_label.style.text_color = 'red'
                self.d.player_value_label.style.text_color = 'green'
                print("Player wins with a Blackjack!")
                return 1.5
        
        if player_hand.value > 21:
            print("Player busts!")
            self.d.dealer_value_label.style.text_color = 'green'
            return -1
        if self.dealer.value > 21:
            self.d.player_value_label.style.text_color = 'green'
            print("Dealer busts!")
            return 1

        
        
        if self.dealer.value == player_hand.value:
            print("Push!")
            return 0
        
        if self.dealer.value > player_hand.value:
            self.d.dealer_value_label.style.text_color = 'green'
            self.d.player_value_label.style.text_color = 'red'
            print("Dealer wins!")
            return -1
        
        if player_hand.value > self.dealer.value:
            self.d.dealer_value_label.style.text_color = 'red'
            self.d.player_value_label.style.text_color = 'green'
            print("Player wins!")
            return 1

    def play_round(self):
        # setup
        self.d.reset_colors()
        self.player = Hand()
        self.dealer = Hand()

        self.player_split_hand = None
        self.player_doubled_down = False
        self.other_hand = None

        # dealer first card
        self.dealer_first_card()
        
        # player
        self.player_turn()

    def chance_to_bust(self, hand):
        maximum_draw = 21 - hand.get_lowest_value()
        vals = self.deck.get_value_counts()
        vals[1] = vals[11]
        vals = vals.drop(11)
        bust_amount = vals[vals.index > maximum_draw].values.sum()
        return bust_amount / len(self.deck)

